In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

import sys
sys.path.append('/content/gdrive/My Drive/workspace')

%cd 'gdrive/My Drive/workspace'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/workspace


In [2]:
import LBP
import preprocessing as prep

import numpy as np
from collections import Counter
import matplotlib.pyplot as plt


import dlib

from PIL import Image


from importlib import reload

_ = reload(prep)

from sklearn.model_selection import train_test_split

from FaceRecognizer import FaceRecognizer

import helpers
from os.path import join
reload(helpers)
import math

Using TensorFlow backend.


In [0]:
predictor_path = 'shape_predictor_68_face_landmarks.dat'
detector_path = 'mmod_human_face_detector.dat'
root_path = 'lfw'
people_txt = 'people.txt'

path_to_weights = 'vgg_face_weights.h5'

In [0]:
cnn_face_detector = dlib.cnn_face_detection_model_v1(detector_path)

sp = dlib.shape_predictor(predictor_path)
vgg = FaceRecognizer(path_to_weights=path_to_weights)

data load start here

In [0]:
df = helpers.load_names(people_txt, 6,20)
names = set(df['name'])
df['image_path'] = df.apply(lambda row: join(root_path, row['name'], row['name']+'_'+str(row['image_num']).zfill(4) + '.jpg'),axis=1)

In [6]:
df.shape, len(names)

((2502, 3), 254)

In [0]:
train, test = train_test_split(df, test_size=0.2, stratify=df['name'])

# rest, test = train_test_split(df, test_size=0.2, stratify=df['name'])
# train, _  = train_test_split(rest, test_size=0.2, stratify=rest['name'])

In [8]:
print(test.shape, train.shape), len(set(train.name)), len(set(test.name))

(501, 3) (2001, 3)


(None, 254, 254)

In [0]:
def load_prep_featurize(imgpath):
    
    img = dlib.load_rgb_image(imgpath)
    funneled_image_arr = prep.cropped_and_aligned(img, cnn_face_detector, sp,method='cnn')
    funneled_image_arr = np.expand_dims(funneled_image_arr, axis=0)
    return vgg.get_features(funneled_image_arr)

In [10]:
train['features'] = train['image_path'].apply(load_prep_featurize)
test['features'] = test['image_path'].apply(load_prep_featurize)

/content/gdrive/My Drive/workspace/preprocessing.py:99: UserWarning: Found 2 faces, using the one with the highest confidence score.
  warn('Found {} faces, using the one with the highest confidence score.'.format(len(dets)))
/content/gdrive/My Drive/workspace/preprocessing.py:99: UserWarning: Found 3 faces, using the one with the highest confidence score.
  warn('Found {} faces, using the one with the highest confidence score.'.format(len(dets)))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

In [0]:
from sklearn import svm
        
    
from sklearn import metrics



In [0]:
clf = svm.SVC(kernel='linear') # Linear Kernel



In [0]:
X_train = np.stack(train['features'].values)
X_test = np.stack(test['features'].values)
y_train = np.array(train['name'].values)
y_test = np.array(test['name'].values)

In [14]:
clf.fit(X_train, y_train)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [0]:
y_pred = clf.predict(X_test)

In [16]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9500998003992016
